In [11]:
import pandas as pd
import numpy as np
import pycmap
from datetime import datetime, timedelta
import pytz
import matplotlib.pyplot as plt
import os
import glob
import plotly.express as px
from astral import Observer
from astral.sun import sun
from astral.sun import sunrise
from astral.sun import sunset
from astral.sun import zenith

In [36]:
df = pd.read_csv('G4_par_wind.csv')
df['time'] = pd.DatetimeIndex(df.time.values)
df.drop(['Unnamed: 0'], axis = 1, inplace = True)
df

,time,licor_par,cruise,pop,n_per_uL,c_per_uL,lat,lon,temp,salinity,par,diam_med,hour,round_lat,round_lon,scalar
0,2021-11-19 14:00:00+00:00,NaN,TN397_740,prochloro,33.649952,0.693725,NaN,NaN,NaN,NaN,25.798556,0.020616,14,NaN,NaN,NaN
1,2021-11-19 15:00:00+00:00,NaN,TN397_740,prochloro,30.768391,0.620610,NaN,NaN,NaN,NaN,257.557450,0.020170,15,NaN,NaN,NaN
2,2021-11-19 16:00:00+00:00,401.968333,TN397_740,prochloro,47.472140,1.050904,31.164511,-119.758154,17.764602,33.327503,616.760250,0.022137,16,31.0,-120.0,5.600031
3,2021-11-19 17:00:00+00:00,486.087167,TN397_740,prochloro,59.103647,1.471909,31.047502,-119.963323,17.614825,33.200625,909.702550,0.024904,17,31.0,-120.0,5.205841
4,2021-11-19 18:00:00+00:00,891.885167,TN397_740,prochloro,73.904420,1.901776,30.962975,-120.109363,17.633781,33.158950,1619.012500,0.025733,18,31.0,-120.0,5.037268
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
567,2021-12-15 00:00:00+00:00,NaN,TN397_740,prochloro,242.902922,9.390323,19.487649,-156.519662,26.361228,34.824375,2261.087000,0.038659,0,19.5,-156.5,4.035211
568,2021-12-15 01:00:00+00:00,NaN,TN397_740,prochloro,233.212591,8.692187,19.614395,-156.613481,26.368551,34.817905,1681.674300,0.037272,1,19.5,-156.5,3.799886
569,2021-12-15 02:00:00+00:00,NaN,TN397_740,prochloro,267.826398,11.357847,19.741617,-156.711600,26.324589,34.814253,1087.486050,0.042407,2,19.5,-156.5,3.223283
570,2021-12-15 03:00:00+00:00,NaN,TN397_740,prochloro,287.264910,13.591498,19.877727,-156.816770,26.184796,34.816501,163.322400,0.047313,3,20.0,-157.0,3.791519


In [16]:
sat_par = pd.read_csv('/Users/ceg/Desktop/kristine/SACNAS-2023-project/sat_par.csv')
sat_par['Date'] = pd.DatetimeIndex(sat_par.Date.values)
sat_par.drop(['Unnamed: 0', 'cruise', 'PAR'], axis = 1, inplace = True)
sat_par

,Date,sat_par
0,2021-11-19 03:00:00+00:00,0.017180
1,2021-11-19 14:00:00+00:00,20.648454
2,2021-11-19 15:00:00+00:00,161.488521
3,2021-11-19 16:00:00+00:00,386.708677
4,2021-11-19 17:00:00+00:00,576.539448
...,...,...
559,2021-12-14 18:00:00+00:00,942.780117
560,2021-12-14 19:00:00+00:00,1293.382560
561,2021-12-14 20:00:00+00:00,1381.370285
562,2021-12-14 21:00:00+00:00,1060.692890


In [26]:
df1 = pd.read_csv('Gradients4_TN397_Optics_LISST_ACS_ECO.csv')
df1['time'] = pd.DatetimeIndex(df1.time.values)
df2 = df1.groupby([pd.Grouper(key = 'time', freq = 'H')]).agg({'chla_eco': 'mean'}).reset_index()
g4_chloro = df2[df2['chla_eco'].notnull()]
g4_chloro

,time,chla_eco
0,2021-11-19 17:00:00+00:00,0.234573
1,2021-11-19 18:00:00+00:00,0.163371
2,2021-11-19 19:00:00+00:00,0.107802
3,2021-11-19 20:00:00+00:00,0.146960
4,2021-11-19 21:00:00+00:00,0.139737
...,...,...
593,2021-12-14 10:00:00+00:00,0.115329
594,2021-12-14 11:00:00+00:00,0.107005
595,2021-12-14 12:00:00+00:00,0.100188
596,2021-12-14 13:00:00+00:00,0.116568


In [27]:
chla_satpar = pd.merge(g4_chloro, sat_par, left_on = 'time', right_on = 'Date', how = 'inner')
chla_satpar.drop(['Date'], axis = 1, inplace = True)
chla_satpar

,time,chla_eco,sat_par
0,2021-11-19 17:00:00+00:00,0.234573,576.539448
1,2021-11-19 18:00:00+00:00,0.163371,1088.754558
2,2021-11-19 19:00:00+00:00,0.107802,1286.427594
3,2021-11-19 20:00:00+00:00,0.146960,1217.701810
4,2021-11-19 21:00:00+00:00,0.139737,1091.592987
...,...,...,...
501,2021-12-13 18:00:00+00:00,0.077962,1210.487611
502,2021-12-13 19:00:00+00:00,0.068782,1266.557619
503,2021-12-13 20:00:00+00:00,0.050990,1365.744348
504,2021-12-13 21:00:00+00:00,0.037757,1409.942455


In [37]:
g4_data = pd.merge(df, chla_satpar, left_on = 'time', right_on = 'time', how = 'inner')
g4_data

,time,licor_par,cruise,pop,n_per_uL,c_per_uL,lat,lon,temp,salinity,par,diam_med,hour,round_lat,round_lon,scalar,chla_eco,sat_par
0,2021-11-19 17:00:00+00:00,486.087167,TN397_740,prochloro,59.103647,1.471909,31.047502,-119.963323,17.614825,33.200625,909.702550,0.024904,17,31.0,-120.0,5.205841,0.234573,576.539448
1,2021-11-19 18:00:00+00:00,891.885167,TN397_740,prochloro,73.904420,1.901776,30.962975,-120.109363,17.633781,33.158950,1619.012500,0.025733,18,31.0,-120.0,5.037268,0.163371,1088.754558
2,2021-11-19 19:00:00+00:00,1092.274333,TN397_740,prochloro,60.774550,1.420525,30.799496,-120.382978,18.038000,33.079418,1876.892611,0.023374,19,31.0,-120.5,5.219913,0.107802,1286.427594
3,2021-11-19 20:00:00+00:00,1002.763667,TN397_740,prochloro,55.664875,1.528290,30.685723,-120.577657,17.635111,33.002106,1953.495900,0.027455,20,30.5,-120.5,5.233759,0.146960,1217.701810
4,2021-11-19 21:00:00+00:00,922.692167,TN397_740,prochloro,63.412060,1.730710,30.605379,-120.714316,17.676354,33.059641,2278.979714,0.027293,21,30.5,-120.5,5.602349,0.139737,1091.592987
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
468,2021-12-13 18:00:00+00:00,688.223000,TN397_740,prochloro,241.180751,8.157731,14.772220,-152.864218,26.781531,33.951840,1866.283750,0.033824,18,15.0,-153.0,9.271543,0.077962,1210.487611
469,2021-12-13 19:00:00+00:00,1092.185333,TN397_740,prochloro,245.492471,8.885463,14.777021,-152.867004,26.787131,33.953178,2020.028100,0.036194,19,15.0,-153.0,9.141849,0.068782,1266.557619
470,2021-12-13 20:00:00+00:00,1320.478000,TN397_740,prochloro,244.086001,9.166140,14.812232,-152.894542,26.764932,33.947510,2178.220650,0.037553,20,15.0,-153.0,9.028236,0.050990,1365.744348
471,2021-12-13 21:00:00+00:00,1379.093846,TN397_740,prochloro,254.518232,10.394303,14.977258,-153.025971,26.662743,33.941427,2250.400850,0.040839,21,15.0,-153.0,8.841401,0.037757,1409.942455
